In [ ]:
from pylab import *

import skimage
from skimage import color, io
import numpy as np
import scipy

In [ ]:
img= io.imread('sekvence/girl_ht.tif')


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Iz vremenskog domena ne moze mnogo da se uradi, vršićemo filtriranje u frekvencijskom domenu.

In [ ]:
img_pad_fft = fftshift(fft2(img_pad))

input_title = 'frekvencijski domen'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(log(1 + np.abs(img_pad_fft)), cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)


Ako stavimo sliku na 2.5 stepen mozemo istaci pikove.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

amplf = log(1 + np.abs(img_pad_fft))**2.5

axes.imshow(amplf, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Sad treba da izvrsimo detekciju poovljenih kopija spektra (posledica tačkastog prikaya u vremenskom domenu).

In [ ]:
import numpy as np
from scipy.ndimage.filters import maximum_filter


T = 16.5

amplf =np.abs( img_pad_fft )
#amplf = skimage.exposure.equalize_adapthist(amplf, [Px/64, Py/64], nbins=256)*scale
amplf = log(1 + 10*np.abs(amplf))
detected_peaks = maximum_filter(amplf, size = 10)

print(np.max(detected_peaks))

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,16), dpi=120)
detected_peaks[detected_peaks<=T] = 0
detected_peaks[detected_peaks > T] = 1
ax[0].imshow(amplf, cmap = 'gray'); ax[0].set_title('Thresholded spectrum', fontsize=14); ax[0].set_axis_off();
#ax[0].scatter(c_x, c_y, s=1, c='red', marker='o')
ax[1].imshow(detected_peaks, cmap = 'gray'); ax[1].set_title('Detected peaks', fontsize=14); ax[1].set_axis_off();



In [ ]:




y, x = np.where(detected_peaks==(1))


mean = 0
k = 0
mean_x = np.array([]);
x = sort(x)
for i in x:
    mean+=i
    k += 1
    
    if(mean*1.06/k < i):
        mean -= i
        k -= 1
        mean_x = np.append(mean_x, int(mean//k))
        
        mean = i
        k = 1
if(k!=1):
    mean_x = np.append(mean_x, int(mean//k))
    
mean = 0
k = 0
mean_y = np.array([]);
y = sort(y)
for i in y:
    mean+=i
    k += 1
    
    if(mean*1.08/k < i):
        mean -= i
        k -= 1
        mean_y = np.append(mean_y, int(mean//k))
        
        mean = i
        k = 1
if(k!=1):
    mean_y = np.append(mean_y, int(mean//k))
    
print(mean_x)
print(mean_y)

c_x = (Px-1)//2
c_y = (Py-1)//2

diff_mean_x = np.diff(mean_x)
diff_mean_y = np.diff(mean_y)


print(diff_mean_x)
print(diff_mean_y)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)
ax.imshow(detected_peaks, cmap = 'gray'); ax.set_title('Detected peaks', fontsize=14); ax.set_axis_off();
for i in mean_x:
    for j in mean_y: 
            ax.scatter(i, j, s=1, c='red', marker='o')

Na osnovu pozicije detektovanih pikova otkrili smo periodičnost po x i y osi, na osnovu ovoga možemo isprojektovati niz notch filtara. Periodičnost je očekivana zbog implicitnog upscaling-a

In [ ]:
f_x = 151
f_y = 73

curr = c_x - f_x
mesh_x = np.array([]).astype(int8)
while curr > 0 :
    mesh_x = np.append(mesh_x, curr)
    curr = curr - f_x

curr = c_x + f_x
while curr < Px:
    mesh_x = np.append(mesh_x, curr)
    curr = curr + f_x

curr = c_y - f_y
mesh_y = np.array([]).astype(int8)
while curr > 0 :
    mesh_y = np.append(mesh_y, curr)
    curr = curr - f_y

curr = c_y + f_y
while curr < Py:
    mesh_y = np.append(mesh_y, curr)
    curr = curr + f_y
mesh_y = np.append(mesh_y, c_y)
mesh_x = np.append(mesh_x, c_x)



In [ ]:


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(log(1 + np.abs(img_pad_fft)), cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)
for i in mesh_x:
    for j in mesh_y: 
        axes.scatter(i, j, s=1, c='red', marker='o')

Vidimo da su aproksimacije pozicije kopija dovoljno dobre za potrebe filtriranja na niskim učestanostima, međutim na visokim učestanostima dolazi do razmazivanja kopija i njihovog izmeštanja, zato ćemo primeniti jedan NF radijusa $r^2 = \frac{300}{3}$

In [ ]:





def lpfilter(Ny, Nx, sigma, off_x = 0, off_y = 0):
    
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2

    
    X, Y = meshgrid(x, y)
    
    D = np.sqrt(np.square(X - off_x) + np.square(Y- off_y))
    
    filter_mask = exp(-np.square(D)/(2*np.square(sigma)))
    filter_mask2 = np.ones((Ny, Nx))
    filter_mask2 [(D >= 5*sigma)] = 0
    
    return filter_mask, filter_mask2
        

c_x = (Px-1)//2
c_y = (Py-1)//2
    

    
    
radius = 300/3

lp_filt_freq, cutoff_filt = lpfilter( Py, Px, radius)

img_pad_fft_filt = log(1 + np.abs(img_pad_fft)*lp_filt_freq*cutoff_filt)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img_pad_fft_filt, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)
for i in mesh_x:
    for j in mesh_y: 
        if((i - c_x)**2 + (j-c_y)**2 <=550**2):
            axes.scatter(i, j, s=1, c='red', marker='o')

Sledi formiranje notch filtara, za notch filtre usvajamo $r = 35$ Moguće je isprojektovati notch filtre za sve smetnje (npr. možemo da usvojimo dinamički radius koji raste sa povećanjem rastojanja od centra slike), međutim to značajno uvećava broj potrebnih filtara. Kombinacija NF i manjeg broja notch filtara predstavlja kompromis između složenosti algoritma i kvaliteta filtriranja.

In [ ]:


y = np.arange(0,Py) - (Py-1)/2 
x = np.arange(0,Px) - (Px-1)/2
X, Y = meshgrid(x, y)
r = 35
n = 10
filter_mask = zeros([Py,Px])

for idxi, i in enumerate(mesh_x):
    for idxj, j in enumerate(mesh_y):
        if((i - c_x)**2 + (j-c_y)**2 <= 550**2):
            if not(np.abs(i - c_x) < 10 and np.abs(j-c_y)<10):
        
                y0 = y - j -(Py-1)/2 
                x0 = x - i - (Px-1)/2
                X0, Y0 = meshgrid(x0, y0)
                D0 = np.sqrt(np.square(X0) + np.square(Y0))
                #filter_mask = filter_mask + exp(-np.square(D0)/(2*np.square(r)))
                #filter_mask[(D0<=r)] = 1
                y0c = y +j - (Py-1)/2 
                x0c = x +i - (Px-1)/2
        
                X0c, Y0c = meshgrid(x0c, y0c)
        
                D0c = np.sqrt(np.square(X0c) + np.square(Y0c))
    
       
                #filter_mask = filter_mask + \
                 #         exp(-np.square(D0)/(2*np.square(r))) + \
                 #       exp(-np.square(D0c)/(2*np.square(r)))
                filter_mask = filter_mask + \
                          1/(1+(D0/r)**(2*n)) + 1/(1+(D0c/r)**(2*n))
                
filter_mask = 1-filter_mask

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(log(1 + np.abs(img_pad_fft*lp_filt_freq*filter_mask)), cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Filtriranje je uspešno izvršeno, ostaje nam da primenimo filtriranje na ulaznu sliku. 

In [ ]:
ilter_maskinput_title = 'Rezultat filtriranja'

img_fft_filt = img_pad_fft*lp_filt_freq*filter_mask

img_filt = real(ifft2(ifftshift(img_fft_filt)))

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,16), dpi=120)
ax[0].imshow(img, cmap = 'gray'); ax[0].set_title('Ulazna slika', fontsize=14); ax[0].set_axis_off();
ax[1].imshow(img_filt, cmap = 'gray'); ax[1].set_title('Rezultat filtriranja', fontsize=14); ax[1].set_axis_off();






Možemo primetiti tragove kružića blizu ivica, što nam ukazuje na to da su ti artefakti na višim učestanostima, taj problem bi bio smanjen da smo filtrirali sve kopije spektra na višim učestanostima.

# Zadatak 2

In [ ]:
img= io.imread('sekvence/road_blur.png')


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Prvi korak je estimacija degradacione fukcije, to ćemo učiniti na osnvu spektra jednog sloja slike.

In [ ]:
input_title = 'detalj'

R = img[:, :, 0];
G = img[:, : , 1];
B = img[:, :, 2];


R_fft = fftshift(fft2(R))

input_title = 'frekvencijski domen'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(log(1 + np.abs(R_fft)), cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

In [ ]:

[Ny, Nx] = shape(R)
radius = 25
lp_filt_freq, cutoff_filt = lpfilter( Ny, Nx, radius)
amplf = log(1 + np.abs(R_fft*cutoff_filt))        
varx = (radius*3)**2
radius = 15
lp_filt_freq, cutoff_filt = lpfilter( Ny, Nx, radius)
amplf2 = log(1 + np.abs(R_fft*cutoff_filt))   
vary = (radius*3)**2

print(vary, varx)


fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,16), dpi=120)

ax[0].imshow(amplf, cmap = 'gray'); ax[0].set_title('frekvencijski domen', fontsize=14); ax[0].set_axis_off();
#ax[0].scatter(c_x, c_y, s=1, c='red', marker='o')
ax[1].imshow(amplf2, cmap = 'gray'); ax[1].set_title('frekvencijski domen', fontsize=14); ax[1].set_axis_off();


Na osnovu spektra možemo da primetimo da je degradacija u obiku elipse, na x osi standardna devijacija iznosi ~75, a na y osi standardna devijacija iznosi ~45

Zamućenje se može modelovati degradacionom funkcijom oblika: $H(u, v) = e^{-0.5[\frac{u^2}{\sigma^2_u}+\frac{v^2}{\sigma^2_v}]}$
Filtriranje vršimo korišćenjem Constrained LS filtra, kao rezultat minimizuje vrednost 
laplasijana slike, tj. očekujemo da slika u određenij meri bude glatka.


In [ ]:
def constrained_LS(img, gamma, H, k):
    [Ny, Nx] = shape(img)
    h_lap = [[1,  1, 1],
         [1, -8, 1],
         [1,  1, 1]]

    h_lap_p = zeros([Ny, Nx])
    h_lap_p[0:3,0:3] = h_lap

    P = np.fft.fftshift(np.fft.fft2(h_lap_p))
    
    F = np.conjugate(H)/(np.abs(H)**2 + gamma*np.abs(P)**2)*np.fft.fftshift(np.fft.fft2(img))
    
    return np.real(np.fft.ifft2(np.fft.ifftshift(F*k)))


def weiner(img, sigma, H):
    [Ny, Nx] = shape(img)
    F = np.empty((Ny, Nx), dtype = 'complex')
    S = np.fft.fftshift(np.fft.fft2(img))
    W = (np.abs(H)**2)/(np.abs(H)**2 + sigma**2)
    F = S*W/H
    return np.real(np.fft.ifft2(np.fft.ifftshift(F)))
 
[Ny, Nx] = shape(R)


    
if (Ny%2 == 0):
    y = np.arange(0,Ny) - Ny/2 + 0.5
else:
    y = np.arange(0,Ny) - (Ny-1)/2
    
if (Nx%2 == 0):
    x = np.arange(0,Nx) - Nx/2 + 0.5
else:
    x = np.arange(0,Nx) - (Nx-1)/2


U, V = meshgrid(x, y)
D0 = np.sqrt(np.square(U) + np.square(V))

R_fft = np.fft.fftshift(np.fft.fft2(R))

gamma = 0.00001
varx = 5400
vary = 1400

H = np.exp(-(np.square(U)/varx +  np.square(V)/vary)/2)

input_title = 'detalj'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

R1  =constrained_LS(R/255, gamma, H, 1)
R1[R1>=1] = 1
R1[R1<0] = 0

G1  =constrained_LS(G/255,gamma, H, 1)
G1 = G1
G1[G1>=1] = 1
G1[G1<0] = 0

B1 = constrained_LS(B/255, gamma, H, 1)
B1[B1>=1] = 1
B1[B1<0] = 0

temp = np.dstack((R1,G1, B1))
axes.imshow(log(1+abs(H)), cmap = 'gray'); axes.set_axis_off(); axes.set_title('Degradacija u frekvencijskom domenu')


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)


R2_fft = fftshift(fft2(R1))
axes.imshow(temp, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)
axes.imshow(log(1+abs(np.fft.fftshift(np.fft.fft2(R1)))), cmap = 'gray'); axes.set_axis_off(); axes.set_title('Filtrirana slika u frekv domenu')


Nakon finog podešavanja varijansi na x i y osi došli smo do zadovoljavajućeg rezultata. Sledi poboljšanje kontrasta.

In [ ]:
N = 16
clip =0.02
r1 = skimage.exposure.equalize_adapthist(R1, [Ny/N, Nx/N], clip_limit=clip, nbins=256)
b1 = skimage.exposure.equalize_adapthist(B1, [Ny/N, Nx/N], clip_limit=clip, nbins=256)
g1 = skimage.exposure.equalize_adapthist(G1, [Ny/N, Nx/N], clip_limit=clip, nbins=256)

temp = np.dstack((r1, g1, b1))
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)
axes.imshow(temp, cmap = 'gray'); axes.set_axis_off(); axes.set_title('Poboljšan kontrast')

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)
axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title('Ulazna slika')

Možemo videti da je kvalitet slike značajno poboljšan i da smo u velikoj meri uklonili zamućenje.

# Zadatak 3

In [ ]:
import random

def quickselect_median(l, pivot_fn = random.choice):
    if len(l) % 2 == 1:
        return quickselect(l, len(l)//2, pivot_fn)
    else:
        return 0.5*(quickselect(l, len(l)/2 - 1, pivot_fn) +
                    quickselect(l, len(l)/2, pivot_fn))
    
def quickselect(l, k, pivot_fn): 
    if len(l) == 1:
        assert k == 0
        return l[0]
    pivot = pivot_fn(l)
    
    lows = [el for el in l if el < pivot]
    highs = [el for el in l if el > pivot]
    pivots = [el for el in l if el == pivot]
    
    if k < len(lows):
        return quickselect(lows, k, pivot_fn)
    elif k < len(lows) + len(pivots):
        return pivots[0]
    else:
        return quickselect(highs, k - len(lows) -len(pivots), pivot_fn)
    


Algoritam za pronalaženje medijane u približno linearnom vremenu. Funkcija je sledeća:
1) Započinjemo pretragu od pivot elementa
2) Delimo listu na 2 pod grupe: Elementi veći od pivota i elementi manji od pivota
3) Medijana će se nalaziti u jednom od dva skupa, to mora biti veći skup. Potragu nastavljamo na većem skupu gde kao pivot usvajamo središnji element

In [ ]:
test = [1, 1, 1, 2, 3, 3, 4]
print( quickselect_median(test))
test = [-100, 1, 1, 3, 36, 336, 4000]
print( quickselect_median(test))

Na osnovu test primera smo se uverili da algoritam dobro radi, prednost ovog pristupa je to što ne zahteva sortiranje, složenost sortiranja je u opštem slučaju n log n.

In [ ]:
def dos_adaptmedian(s, max_order = 3):
    """
    Funkcija vrši filtriranje ulazne slike, uklanjanje šum uz maksimalno očuvanje ivica. 
    
    Parametar max_oder određuje maksimalnu veličinu prozora na kome se vrši pretraga,
    prozor je dimencija max_order x max_order
    
    Ako piskel nije implusne priode njegova vrednost se ne menja.
    Ako piksel jeste impulsne prirode njegova vrednost je medijana lokalnog prozora, 
    ako je i medijana impulsne prirode dimenzije prozora se uvecavaju sve dok se ne 
    pronadje medijana koja nije impulsne prirode ili dostigne velicina prozora max_order
    
    Pre samog procesa filtriranja vrši se proširenje slike kopijama piksela sa ivice, tj. obrada se efektivno
    vrši nad slikom dimenzije (Ny + max_order)x(Nx + max_order)x1
     
    Ulazi
      s - matrica dimenzija NxMx1. Ulazna slika. Format: unit8/float64
      max_order - najveća dimenzija maske, pozitivan ceo broj
      
    Izlazi
      img_pad - realna matrica dimenzija NxMx1. Filtrirana slika. Format: unit8/float64
      
      U slučaju greške napušta se funckija i ispisuje se poruka o grešci.
    
    Primer
        lena = skimage.img_as_float(imread('.../lena.png'))
        imgR = lena[:,:,0];
        
        x,y = imgR.shape
        
         temp = np.array(imgR)
         
    
        for m in range(x):
            for n in range(y):
                rdn = np.random.random()
                rnd2 = np.random.random()
                if rdn < pepper and rnd2<0.5:
                    temp[m, n] = 0
                elif rdn < salt and rnd2>=0.5:
                    temp[m, n] = 1
                else:
                    temp[m, n] = imgV[m, n]
        
        imgR_filt =  dos_adaptmedian(temp, 5)
        
        

        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120); ax = axes.ravel();
        ax[0].imshow(temp, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); 
        ax[1].imshow(imgR_filt, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off();

        
    Reference
        https://en.wikipedia.org/wiki/Median_filter
        https://rcoh.me/posts/linear-time-median-finding/
    """
    
    max_order = int(max_order)
    start_order = 1
    if(max_order < 1):
        max_order = 1
   

    
    if not (s.dtype == np.float64 or s.dtype == np.uint8):
        print('Error: only float64 and uint8 are supported')
        return
       
    N_extra = max_order*2
    [Ny, Nx] = shape(s)
    Px = Nx + 2*N_extra; Py = Ny + 2*N_extra;
    
    img_pad = zeros([Py, Px], dtype = s.dtype)
    img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx] = s
    
    
    for i in range(Nx):
        for j in range(Ny):
            increase_order = True
            order = start_order - 1
            check_xy = True
            
            x = i + N_extra
            y = j + N_extra
            
            while increase_order: 
                order = order + 1
                k = order
                
                current = img_pad[y - k : y + k  + 1, x - k : x + k + 1].flatten(order='C')
                max_px = np.max(current)
                min_px = np.min(current)
                med_px = quickselect_median(current)
                
                if (s[j, i] - min_px > 0) and (s[j, i] - max_px < 0):
                    increase_order = False
                    img_pad[N_extra + j, N_extra + i] = s[j, i]
                    continue
                 
                if (med_px - min_px > 0) and (med_px - max_px < 0):
                    "Proveravamo da li je med zapravo impuls"
                    increase_order = False
                if(order >= max_order): 
                    check_xy = False
                    increase_order = False
                    img_pad[N_extra + j, N_extra + i] = med_px
                    
            if(check_xy):
                if (s[j, i] - min_px > 0) and (s[j, i] - max_px < 0):
                    img_pad[N_extra + j, N_extra + i] = s[j, i]
                else:
                    img_pad[N_extra + j, N_extra + i] = med_px 
                    "menja s[x, y] samo ako je impulsne prirode"
              
    return img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx]

Funkcija vrši proširenje ulazne slike nulama. Što se tiče samog algoritma povećava se lokalna okolina na kojoj se vrši pretraga sve dok se ne pronađe lokalna okolina gde medijana nije minimum-maksimum. Zatim se vrednost piksela, koji se trenutno obrađuje, menja samo ako je sam piksel impulsne prirode. Ako piksel nije impulsne prirode preskače se korak pretrage i njegova vrednost se ne menja.

In [ ]:
percent = [0.2, 0.4, 0.6, 0.8]

img= io.imread('sekvence/lena.png')
imgV = color.rgb2hsv(img)[:,:,2]



x,y = imgV.shape
img_list = []

for i in percent:
    salt = i
    pepper = i
    
    temp = np.array(imgV)
    
    for m in range(x):
        for n in range(y):
            rdn = np.random.random()
            rnd2 = np.random.random()
            if rdn < pepper and rnd2<0.5:
                temp[m, n] = 0
            elif rdn < salt and rnd2>=0.5:
                temp[m, n] = 1
            else:
                temp[m, n] = imgV[m, n]
    img_list.append(temp)


In [ ]:
j = 0

for i in img_list:
    img_filt =  dos_adaptmedian(i)
    
    
    
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,16), dpi=120)
    ax[0].imshow(i, cmap = 'gray'); ax[0].set_title(f"Ulazna slika P={percent[j]}", fontsize=14); ax[0].set_axis_off();
    ax[1].imshow(img_filt, cmap = 'gray'); ax[1].set_title('Rezultat filtriranja', fontsize=14); ax[1].set_axis_off();
    j += 1

In [ ]:
j = 0
for i in img_list:
    img_filt =   skimage.filters.median(i)
    
    
    
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,16), dpi=120)
    ax[0].imshow(i, cmap = 'gray'); ax[0].set_title(f"Ulazna slika P={percent[j]}", fontsize=14); ax[0].set_axis_off();
    ax[1].imshow(img_filt, cmap = 'gray'); ax[1].set_title('Rezultat filtriranja', fontsize=14); ax[1].set_axis_off();
    j += 1

Kao što je i očekivano filtar dobro radi za razumne vrednosti impulsnog šuma, za veće vrednosti dolazi do pojave fraktalnih artefaktka koji zapravo predstavljaju vrednosti piksela koje nisu zašumljeni (Tj. kakav god filtar bio ne može da stvori informaciju koja ne postoji). Za najveći intezitet šuma vidimo da je filtar udario gornju granicu tokom pretrage okoline za medijanu, pa kao posledicu imamo da je slika praktično neizmenjena. 

Poređenjem sa ugrađenom funkcijom gde se ne vrši adaptivna obrada vidimo da adaptivni medijan bolje radi.